# Test Valuation Engines

This notebook walks through:

1. Loading historical SOFR fixings
2. Building a simple two-index yield curve
3. Valuing various products (bullet, IBOR cashflow, OIS cashflow, futures, swaps)

---

## 1. Imports & Setup

In [1]:
import sys, os

# compute the parent of tests → FixedIncomeLib
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

print("Added to sys.path:", repo_root)

Added to sys.path: c:\Users\neels\OneDrive\Desktop\Capstone_Project\FixedIncomeLib


In [2]:
import pandas as pd
from data import DataCollection, Data1D, build_yc_data_collection
from date import Date
from yield_curve import YieldCurve
from valuation import IndexManager
from valuation import ValuationEngineRegistry
from product import (
    ProductBulletCashflow,
    ProductIborCashflow,
    ProductOvernightIndexCashflow,
    ProductFuture,
    ProductRfrFuture,
    ProductIborSwap,
    ProductOvernightSwap,
    ProductPortfolio )
from builders import create_products_from_data1d
from builders import build_yc_calibration_basket
from product.product_display_visitor import RfrFutureVisitor, OvernightSwapVisitor

## 2. Load SOFR Fixings

In [3]:
IndexManager.instance()

## 3. Build Dummy Yield Curve

In [4]:
MARKET_DF = pd.DataFrame(
    [
        ["RFR FUTURE","SOFR-FUTURE-3M","2025-09-24 x 2025-12-24", 95.70],
        ["RFR FUTURE","SOFR-FUTURE-3M","2025-12-24 x 2026-03-24", 95.80],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-03-24 x 2026-06-24", 95.90],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-06-24 x 2026-09-24", 96.00],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-09-24 x 2026-12-24", 96.08],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2026-12-24 x 2027-03-24", 96.16],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-03-24 x 2027-06-24", 96.24],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-06-24 x 2027-09-24", 96.32],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-09-24 x 2027-12-24", 96.38],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2027-12-24 x 2028-03-24", 96.44], 
        ["RFR FUTURE","SOFR-FUTURE-3M","2028-03-24 x 2028-06-24", 96.50],  
        ["RFR FUTURE","SOFR-FUTURE-3M","2028-06-24 x 2028-09-24", 96.55],  
        
        ["RFR SWAP","USD-SOFR-OIS","1Y",  0.0395],
        ["RFR SWAP","USD-SOFR-OIS","2Y",  0.0380],
        ["RFR SWAP","USD-SOFR-OIS","3Y",  0.0372],
        ["RFR SWAP","USD-SOFR-OIS","4Y",  0.0368],
        ["RFR SWAP","USD-SOFR-OIS","5Y",  0.0365],
        ["RFR SWAP","USD-SOFR-OIS","6Y",  0.0371],
        ["RFR SWAP","USD-SOFR-OIS","7Y",  0.0374],
        ["RFR SWAP","USD-SOFR-OIS","8Y",  0.0380],
        ["RFR SWAP","USD-SOFR-OIS","9Y",  0.0383],
        ["RFR SWAP","USD-SOFR-OIS","10Y", 0.0386],  
        ["RFR SWAP","USD-SOFR-OIS","15Y", 0.0395],
        ["RFR SWAP","USD-SOFR-OIS","20Y", 0.0405],
        ["RFR SWAP","USD-SOFR-OIS","25Y", 0.0412],
        ["RFR SWAP","USD-SOFR-OIS","30Y", 0.0419],  
        ["RFR SWAP","USD-SOFR-OIS","40Y", 0.0423],
        ["RFR SWAP","USD-SOFR-OIS","50Y", 0.0428],
        ["RFR SWAP","USD-SOFR-OIS","60Y", 0.0432],
    ],
    columns=["DATA TYPE","DATA CONVENTION","AXIS","VALUE"],
)


In [5]:
data_objs, dc = build_yc_data_collection(MARKET_DF)

In [6]:
build_methods = [{
    "TARGET": "SOFR-1B",
    "REFERENCE": None,
    "INSTRUMENTS": ["SOFR-FUTURE-3M", "USD-SOFR-OIS"],
    "INTERPOLATION METHOD": "PIECEWISE_CONSTANT",
    "DEBUG": True
}]


In [7]:
yc = YieldCurve("2025-09-24", dc, build_methods)
print("Curve components:", yc.components.keys())


[YC DEBUG] Calibrated IFRs at pillars
idx  pillar           t (yrs)      theta*         ∫θ du         DF(t)
---------------------------------------------------------------------
  0  2025-12-24    0.2500000000  0.04324308  0.0108107691  0.9892474572
  1  2026-03-24    0.5000000000  0.04178092  0.0212559994  0.9789683172
  2  2026-06-24    0.7500000000  0.04169295  0.0316792369  0.9688172930
  3  2026-09-24    1.0000000000  0.04068120  0.0418495368  0.9590140660
  4  2026-12-24    1.2500000000  0.03944042  0.0517096428  0.9496045514
  5  2027-03-24    1.5000000000  0.03821681  0.0612638448  0.9405750412
  6  2027-06-24    1.7500000000  0.03825202  0.0708268506  0.9316231881
  7  2027-09-24    2.0000000000  0.03744195  0.0801873373  0.9229434284
  8  2027-12-27    2.2583333333  0.03643576  0.0895999078  0.9142969152
  9  2028-03-24    2.5000000000  0.03581382  0.0982549132  0.9064178165
 10  2028-06-26    2.7555555556  0.03561871  0.1073574711  0.8982045334
 11  2028-09-25    3.00277777


## 4. Value Bullet Cashflow


In [29]:
# Bullet cashflow PV
bullet = ProductBulletCashflow("2028-09-24", "USD", 1000000, "LONG")
ve = ValuationEngineRegistry().new_valuation_engine(
    yc,
    {"FUNDING INDEX": "SOFR-1B"},
    bullet
)
ve.calculateValue()
print("Bullet CF PV:", ve.value)

Bullet CF PV: ['USD', np.float64(890530.779407151)]


## 5. Value IBOR Cashflow

In [ ]:
ibor_cf = ProductIborCashflow(
    startDate="2025-09-27",
    endDate="2025-12-27",
    index="USD-LIBOR-BBA-3M",
    spread=0.0,
    notional=1000000,
    longOrShort="SHORT"
)
ve = ValuationEngineRegistry().new_valuation_engine(yc, {}, ibor_cf)
ve.calculateValue()
print("Ibor CF PV:", ve.value)

AttributeError: 'NoneType' object has no attribute 'isOvernightIndex'

## 6. Value OIS Cashflow

In [34]:
ois_cf = ProductOvernightIndexCashflow(
    effectiveDate="2025-09-24",
    termOrEnd="6M",
    index="SOFR-1B",
    compounding="COMPOUND",
    spread=0.0,
    notional=1000000,
    longOrShort="LONG"
)
ve = ValuationEngineRegistry().new_valuation_engine(
    yc,
    {},
    ois_cf
)
ve.calculateValue()
print("OIS CF PV:", ve.value)

OIS CF PV: ['USD', np.float64(21364.82389100669)]


## 7. Value Futures

In [ ]:
# IBOR Future PV
future = ProductFuture(
    effectiveDate="2025-06-27",
    index="USD-LIBOR-BBA-3M",
    strike=99.5,
    notional=1_000_000,
    longOrShort="LONG"
)
ve = ValuationEngineRegistry().new_valuation_engine(yc, {}, future)
ve.calculateValue()
print("Future PV:", ve.value)

AttributeError: 'NoneType' object has no attribute 'isOvernightIndex'

In [35]:
# RFR Future PV
rfr_future = ProductRfrFuture(
    effectiveDate="2025-09-24",
    termOrEnd="3M",
    index="SOFR-1B",
    compounding="AVERAGE",
    strike=95.9,
    notional=1000000,
    longOrShort="SHORT"
)
ve = ValuationEngineRegistry().new_valuation_engine(
    yc,
    {},
    rfr_future
)
ve.calculateValue()
print("RFR Future PV:", ve.value)

RFR Future PV: ['USD', np.float64(199988.99220539103)]


## 8. Value a Portfolio

In [36]:
# Simple portfolio of bullet + IBOR CF
portfolio = ProductPortfolio([bullet, ois_cf], weights=[0.5, 0.5])
ve = ValuationEngineRegistry().new_valuation_engine(
    yc,
    {"FUNDING INDEX": "SOFR-1B"},
    portfolio
)
ve.calculateValue()
print("Portfolio PV:", ve.value)

Portfolio PV: ['USD', np.float64(911895.6032981577)]


## 9. Value Swaps (IBOR & OIS)

In [ ]:
# IBOR Swap
ibor_swap = ProductIborSwap(
    effectiveDate="2025-06-27",
    maturityDate="2026-06-27",
    frequency="6M",
    iborIndex="USD-LIBOR-BBA-3M",
    spread=0.0,
    fixedRate=0.015,
    notional=1_000_000,
    position="SHORT"
)
ve = ValuationEngineRegistry().new_valuation_engine(
    yc, {"FUNDING INDEX": "SOFR-1B"}, ibor_swap
)
ve.calculateValue()
print("IBOR Swap PV:", ve.value, "Par Rate:", ve.parRateOrSpread())



AttributeError: 'NoneType' object has no attribute 'isOvernightIndex'

In [15]:
# OIS Swap
ois_swap = ProductOvernightSwap(
    effectiveDate="2025-09-24",
    maturityDate="2026-09-24",
    frequency="6M",
    overnightIndex="SOFR-1B",
    spread=0.0,
    fixedRate=0.045,
    notional=1000000,
    position="LONG"
)
ve = ValuationEngineRegistry().new_valuation_engine(
    yc, {"FUNDING INDEX": "SOFR-1B"}, ois_swap
)
ve.calculateValue()
print("OIS Swap PV:", ve.value, "Par Rate:", ve.parRateOrSpread())

OIS Swap PV: ['USD', np.float64(3168.6548769419023)] Par Rate: 0.04172994460177388
